# Transfer learning на примере соревнования kaggle
Данный notebook представляет из себя решение задачи классификации набора изображений грязной и чистой посуды в рамках курса "Нейронные сети и компьютерное зрение" на платформе stepik.ru. 

# Подготовка данных

Подключаем часть необходимых модулей

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torchvision
import matplotlib.pyplot as plt
import time
import copy
import PIL 
import os
from imgaug import augmenters as iaa
import imgaug as ia

Фиксируем генераторы случайных чисел для воспроизводимости результатов.


In [ ]:
import random
import torch

random.seed(3)
np.random.seed(3)
torch.manual_seed(3)
torch.cuda.manual_seed(3)
torch.backends.cudnn.deterministic = True
#!pip install git+https://github.com/aleju/imgaug
#from PIL import *

Смотрим содержимое папки input.

In [ ]:
print(os.listdir("../input"))

Запоминаем путь к данным.

In [ ]:
path_to_data = '../input/plates/plates/'
print(os.listdir(path_to_data))

Перемещаем фотографии для более удобной адресации.

In [ ]:
import shutil 
from tqdm import tqdm

train_dir = 'train'
val_dir = 'val'

# имена классов
class_names = ['cleaned', 'dirty']

for dir_name in [train_dir, val_dir]:
    for class_name in class_names:
        os.makedirs(os.path.join(dir_name, class_name), exist_ok=True)

for class_name in class_names:
    
    source_dir = os.path.join(path_to_data, 'train', class_name)
    for i, file_name in enumerate(tqdm(os.listdir(source_dir))):
        #определяем путь к папке, в которую хотим скопировать файлы
        dest_dir = os.path.join(train_dir, class_name)
        #копируем в нужную папку
        shutil.copy(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))


# Аугментации
Настраиваем изменения фотографий для улучшения точности классификации.

In [ ]:
from torchvision import transforms, models
#преобразования тренеровочной выборки
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    #transforms.CenterCrop(200),
    #transforms.RandomVerticalFlip(),
    transforms.ColorJitter(hue=0.4, brightness=0.5, contrast = 4.0),
    transforms.RandomHorizontalFlip(),
    transforms.Grayscale(3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

#преобразование тестовой выборки
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    #transforms.RandomHorizontalFlip(),
    #transforms.ColorJitter(hue=0.4, brightness=0.5, contrast = 4.0),
    transforms.Grayscale(3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


train_dataset = torchvision.datasets.ImageFolder(train_dir, train_transforms)
#val_dataset = torchvision.datasets.ImageFolder(val_dir, val_transforms) валидационного набора нет

#размер батча
batch_size = 8
#загрузка данных 
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=batch_size)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=batch_size)

## Смотрим на наши преобразования изображений на примере одного батча.

In [ ]:
X_batch, y_batch = next(iter(train_dataloader))
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

In [ ]:
#функция отрисовки картинок одного батча
def show_input(input_tensor, title=''):
    image = input_tensor.permute(1, 2, 0).numpy()
    image = std * image + mean
    plt.imshow(image.clip(0, 1))
    plt.title(title)
    plt.show()
    plt.pause(0.001)

X_batch, y_batch = next(iter(train_dataloader))
for x_item, y_item in zip(X_batch, y_batch):
    show_input(x_item, title=class_names[y_item])

# Загрузка предтренировонной модели и задание её параметров
Загружаем модель и переносим её на gpu, если есть возможность.

In [ ]:
model = models.mobilenet_v2(pretrained = True)
# "Заморозка" сверточных слоев нейронной сети (фиксация весов)
for param in model.parameters():
    param.requires_grad = False

#Добавление полносвязного слоя для бинарной классификации
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, 2)

# Перекладываем вычисления на GPU в случае ее отсутствия на CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

### Определяем параметры нейронной сети.

In [ ]:
lr = 3.0e-4
step_size = 60
gamma = 0.1
num_epochs = 200

In [ ]:
# Определяем функцию потерь 
loss = torch.nn.CrossEntropyLoss()
# Определяем оптимизатор и скорость обучения
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= step_size, gamma=gamma)


### Функция дообучения нейронной сети.

In [ ]:
def train_model(model, loss, optimizer, scheduler, num_epochs):
    #Запоминаем промежуточные результаты для изучения их в дальнейшем
    train_accuracy_history = []
    train_loss_history = []
    for epoch in range(num_epochs):
        #валидация отсутствует
        for phase in ['train']:
            if phase == 'train':
                dataloader = train_dataloader
                model.train()  # Set model to training mode

            running_loss = 0.
            running_acc = 0.

            # Iterate over data.
            for inputs, labels in tqdm(dataloader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # forward and backward
                with torch.set_grad_enabled(phase == 'train'):
                    preds = model(inputs)
                    loss_value = loss(preds, labels)
                    preds_class = preds.argmax(dim=1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss_value.backward()
                        optimizer.step()

                # statistics
                running_loss += loss_value.item()
                running_acc += (preds_class == labels.data).float().mean()
            if phase == 'train':
                
                scheduler.step()
                
            epoch_loss = running_loss / len(dataloader)
            epoch_acc = running_acc / len(dataloader)
            train_accuracy_history.append(epoch_acc)
            train_loss_history.append(epoch_loss)
    return train_accuracy_history, train_loss_history

In [ ]:
history = train_model(model, loss, optimizer, scheduler, num_epochs=num_epochs);

## Предсказание класса для элементов тестовой выборки

In [ ]:
# Копируем тестовые изображения 
# в папку 'test/unknown'
# для правильной работы ImageFolder
test_dir = 'test'
shutil.copytree(os.path.join(path_to_data, 'test'), os.path.join(test_dir, 'unknown'))

In [ ]:
# Модифицируем ImageFolder так, что бы он отображал не только изображение с его меткой, но и путь к изображению
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path
    
test_dataset = ImageFolderWithPaths(test_dir, test_transforms)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=batch_size)

In [ ]:
#фиксируем состояние модели
model.eval()
#создаем лист для предсказаний
test_predictions = []
#создаем лист для путей к изображениям
test_img_paths = []
for inputs, labels, paths in tqdm(test_dataloader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.set_grad_enabled(False):
        preds = model(inputs)
    test_predictions.append(
        torch.nn.functional.softmax(preds, dim=1)[:,1].data.cpu().numpy())
    test_img_paths.extend(paths)
    
test_predictions = np.concatenate(test_predictions)

## Визуализируем часть предсказаний

In [ ]:
inputs, labels, paths = next(iter(test_dataloader))

for img, pred in zip(inputs, test_predictions):
    show_input(img, title=pred)

## Сохраняем предсказания

In [ ]:
submission_df = pd.DataFrame.from_dict({'id': test_img_paths, 'label': test_predictions})

In [ ]:
submission_df['label'] = submission_df['label'].map(lambda pred: 'dirty' if pred > 0.63 else 'cleaned')
submission_df['id'] = submission_df['id'].str.replace('test/unknown/', '')
submission_df['id'] = submission_df['id'].str.replace('.jpg', '')
submission_df.set_index('id', inplace=True)
submission_df.head(n=8)

In [ ]:
submission_df.to_csv('submission.csv')

# Точность классификации составляет 0.95026

### Удаляем созданные нами файлы и директории

In [ ]:
!rm -rf train val test

# Небольшой комментарий к решению
Я перебирал различные аугментации: накладывал шум(https://arxiv.org/pdf/1710.06805.pdf) - приводило к ухудшению качества классификации(скорее всего это связано с архитектурой сети, необходимо не только последний слой считать), пытался сегментировать изображения разными способами, но это приводило к ухудшению (https://docs.opencv.org/master/d7/d4d/tutorial_py_thresholding.html). Также смотрел другие кернелы из других соревнований для улучшения параметров сети.
Менял оптимизатор, learning rate, количество эпох, step_size. Архитектура mobilenetv2 быстро дала хороший резулат, поэтому я решил дальше её оптимизировать. 